### Imports

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import math
import re

### Checagem de diretório

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Git\Estudo_CDados\Atividade_1


### Método de limpeza
***
* Vale a pena ressaltar que fiz este método de limpeza baseado no que o professor Fabio Miranda nos ensinou no semestre passado em Ciência dos Dados (aprendi isso pois estou de DP dessa matéria)
*** 

In [3]:
def cleanup(text):
    texto_sem_rt = text.replace("rt", "")
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = "[!\-:?;.@//]" # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', texto_sem_rt)
    return text_subbed

### Nome da planilha
***
* Essa parte é essencial para a conclusão do projeto, pois é parâmetro de entrada da minha função
***

In [4]:
nome_planilha = 'Spacex.xlsx'
dados = pd.ExcelFile('Spacex.xlsx')

### Separando a planilha em segmento Treinamento e Teste

In [5]:
teste_planilha = pd.read_excel(dados, 'Teste')
teste_reindexado = teste_planilha.set_index("Teste")

### Função de classificação

In [6]:
def transforma_planilha_string(nome_planilha):
    #lendo a planilha
    dados = nome_planilha
    
    ##treinamento
    treinamento = pd.read_excel(dados, 'Treinamento')
    treinamento_reindexado = treinamento.set_index("Treinamento")
    treinamento_relevante = treinamento_reindexado.qual[treinamento_reindexado.qual == 1]
    treinamento_irrelevante = treinamento_reindexado.qual[treinamento_reindexado.qual == 0]
    
    #tranformando em string o treinamento
    string_relevante = ""
    for plvrs in treinamento_relevante.index:
        string_relevante = string_relevante + plvrs
        
    string_irrelevante = ""
    for plvras in treinamento_irrelevante.index:
        string_irrelevante = string_irrelevante + plvras
        
    #utilizando o método de limpeza já implementado no treinamento
    treinamento_relevante_limpo = cleanup(string_relevante.lower())
    treinamento_irrelevante_limpo = cleanup(string_irrelevante.lower())
    
    #separando em relevante e irrelevante o treinamento
    treinamento_relevante_splitado = treinamento_relevante_limpo.split()
    treinamento_irrelevante_splitado = treinamento_irrelevante_limpo.split()
    
    #transformando em panda.Series o treinamento
    #series_treinamento_relevante = pd.Series(treinamento_relevante_splitado)
    #series_treinamento_irrelevante = pd.Series(treinamento_irrelevante_splitado)
        
    return treinamento_relevante_limpo, treinamento_irrelevante_limpo

In [7]:
relevante, irrelevante = transforma_planilha_string(nome_planilha)

### Função de contador de palavras:
***
* Está explicada nos comentários
***

In [8]:
def numero_string_series(relevante,irrelevante,a):
    relevante_series = pd.Series(relevante.lower().split())
    relevante_contado = relevante_series.count()
    
    irrelevante_series = pd.Series(irrelevante.lower().split())
    irrelevante_contado = irrelevante_series.count()
    
    prod_relevante = 1
    prod_irrelevante = 1
    
    lis_relevante_summ = []
    lis_irrelevante_summ = []
    
    a_splitado = a.split()
    tamanho_a_sem_repetição = len(list(set(a_splitado)))
    
    
    for plvs in a_splitado:
        str_relevante_count = relevante_series.str.count(plvs).sum()
        str_irrelevante_count = irrelevante_series.str.count(plvs).sum()

        if str_relevante_count == 0:
            str_relevante_count = str_relevante_count + 1
        else:
            str_relevante_count = str_relevante_count
        if str_irrelevante_count == 0:
            str_irrelevante_count = str_irrelevante_count + 1
        else:
            str_irrelevante_count = str_irrelevante_count
        lis_relevante_summ.append(str_relevante_count)
        lis_irrelevante_summ.append(str_irrelevante_count)
        
    for vles in lis_relevante_summ:
        prod_relevante = prod_relevante * (vles / (relevante_contado + tamanho_a_sem_repetição) )
    for vls in lis_irrelevante_summ:
        prod_irrelevante = prod_irrelevante * (vls / (irrelevante_contado + tamanho_a_sem_repetição) )
    
    return prod_relevante, prod_irrelevante

### Separando a planilha teste pré-categorizada

In [9]:
qualidade = list(teste_planilha.set_index('qual').index)

### Estava dando erro nessas linhas da aba "Teste" da planilha, por isso separei as e coloquei na lista a baixo

In [10]:
deu_errado = [6, 17, 40, 42, 47, 51, 61, 85]

In [11]:
lista_teste_reindexado = list(teste_reindexado.index)

In [12]:
relevante_num = 0
irrelevante_num = 0

relevante_correto = 0
relevante_incorreto = 0

irrelevante_correto = 0
irrelevante_incorreto = 0

lista_teste_reindexado
contador = 0
while contador < len(lista_teste_reindexado):
    if contador not in deu_errado:
        a = lista_teste_reindexado[contador]
        r,i = numero_string_series(relevante,irrelevante,a)
        if r > i:
            relevante_num = relevante_num + 1
            if qualidade[contador] == 1:
                relevante_correto = relevante_correto + 1
            else:
                relevante_incorreto = relevante_incorreto + 1
        elif i > r:
            irrelevante_num = irrelevante_num + 1
            if qualidade[contador] == 0:
                irrelevante_correto = irrelevante_correto + 1
            else:
                irrelevante_incorreto = irrelevante_incorreto + 1
    contador = contador + 1

### Separando a probabilidade, última parte da árvore binária

In [13]:
verdadeiro_positivo = relevante_correto / (relevante_num + irrelevante_num)
falso_positivo = relevante_incorreto / (relevante_num + irrelevante_num)
verdadeiro_negativo = irrelevante_correto / (relevante_num + irrelevante_num)
falso_negativo = irrelevante_incorreto / (relevante_num + irrelevante_num)

In [14]:
soma = verdadeiro_positivo + verdadeiro_negativo + falso_positivo + falso_negativo

### Conclusão do projeto
***
* Parece que o classificado Naive-Bayes está funcionando, mas com alguns pequenos problemas que serão abordados no final deste projeto. Primeiramente gostaria de mostra que assim como no final de uma árvore de probabilidade, a soma da probabilidade nas últimas pontas tem sempre ser igual a 1, a soma dos Positivos (Falso Positivo e Verdadeiro Positivo) com os Negativos (Falso Negativo com Verdadeiro Negativo) também deram exatamente igual a 1.
  * A maior dificuldade do projeto foi um erro que a função "pd.Series.str.count(palavras).sum()" utilizada em         str_relevante_count = relevante_series.str.count(plvs).sum(). Porque às vezes ocorre um erro onde a biblioteca pandas não consegue interpretar o ")" e "(" e isso causa um erro no programa. Por isso eu tive que adicionar manualmente todos os tweets que deram errado em uma lista (com o nome deu_errado)

* A performance foi bem satisfatória para uma primeira tentativa, e como a minha base de dados não foi muito bem selecionada (será abordado também no final deste projeto) o resultado foi um pouco comprometido. Os resultados obtidos foram:
  * Falso Positivo: 21,79%
  * Verdadeiro Positivo: 21,79%
  * Falso Negativo: 29,49%
  * Verdadeiro Negativo: 26,92%
 
* Não tenho parâmetros de verificação para julgar se está ou não dentro do parâmetro necessário para ser considerado um bom Naive-Bayes
  * Mas fiquei feliz com o resultado, pois o programa realiza exatamente pra que ele foi programado

* Como a linguagem do computador é puramente técnica e é interpretada em bits, não há possibilidade do programa julgar que a palavra foi ou não utilizada com tom sarcástico. A dupla negação também entra no mesmo problema, e como está é uma situação dificíl até para seres humanos interpretarem, o programa sobre muito mais para entender
  * Mas se utilizarmos artefatos de "Machine Learning" podemos ensinar um pouco do que é sarcasmo para o programa. Porém ainda seria dificultoso ensiná-lo a interpretar duplas negações, porque mesmo contando o número de "nãos" pode resultar em um total desastre

* Se vocês gostaram deste projeto gostaria de dizer que eu tenho total transparência ao dizer o que eu estou fazendo, como vocês podem analisar os meus critérios de auto-avaliação são bem rigorosos e buscam sempres ser sinceros ao cliente. Mas o diferencial real deste projeto é que ele pode ser utilizado em outras áreas além da classificação de tweet. Com algumas pequenas modificações é possível utilizar o método Naive-Bayes para realizar um filtro anti-spam a serviço da sua empresa (que também pode ser utilizado para fins pessoais)
***

### Pequeno adendo

***
***

* Como a empresa utilizada para a análise foi a "SpaceX" do gênio empresário Elon Musk, as mesmas palavras apareciam frequentemente nas duas categorias ( tanto relevantes quanto irrelevantes). E ocorreu um evento tanto quanto inusitado, a descoberta de um asteróide com um valor gigantesco (alguns falam em quintilhões e outros quadrilhões) acalorou o ânimo dos usuários do twitter e acabou corrompendo a base de dados. Mas o resultado foi bem satisfatório


***
***